Create SOURCE Table (with extra columns)

In [0]:
%sql
CREATE OR REPLACE TABLE databricks_catalog.practice.products_src (
  product_id     INT,
  product_name   STRING,
  price          DOUBLE,
  category       STRING,
  discount       DOUBLE,        -- EXTRA COLUMN
  supplier_code  STRING,        -- EXTRA COLUMN
  load_date      DATE
);


Insert 10 Records (Extra Columns Included)

In [0]:
%sql
INSERT INTO databricks_catalog.practice.products_src VALUES
(1,'Laptop',60000,'Electronics',10,'SUP01','2025-01-01'),
(2,'Mobile',30000,'Electronics',5,'SUP02','2025-01-01'),
(3,'Tablet',20000,'Electronics',8,'SUP03','2025-01-01'),
(4,'TV',45000,'Electronics',12,'SUP01','2025-01-01'),
(5,'Headphones',2000,'Accessories',NULL,'SUP04','2025-01-01'),
(6,'Keyboard',1500,'Accessories',NULL,'SUP04','2025-01-01'),
(7,'Mouse',800,'Accessories',NULL,'SUP04','2025-01-01'),
(8,'Monitor',12000,'Electronics',7,'SUP02','2025-01-01'),
(9,'Printer',18000,'Electronics',6,'SUP05','2025-01-01'),
(10,'Camera',55000,'Electronics',9,'SUP06','2025-01-01');


In [0]:
%sql
select * from databricks_catalog.practice.products_src;

Create TARGET Table (NO extra columns)

In [0]:
%sql
CREATE OR REPLACE TABLE databricks_catalog.spractice.products_tgt (
  product_id   INT,
  product_name STRING,
  price        DOUBLE,
  category     STRING,
  load_date    DATE
);


In [0]:
%sql
select *  from databricks_catalog.spractice.products_tgt;

Read SOURCE & TARGET Schema


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW source_schema AS
SELECT column_name
FROM databricks_catalog.information_schema.columns
WHERE table_schema = 'practice'
  AND table_name = 'products_src';

CREATE OR REPLACE TEMP VIEW target_schema AS
SELECT column_name
FROM databricks_catalog.information_schema.columns
WHERE table_schema = 'spractice'
  AND table_name = 'products_tgt';

Find EXTRA Columns


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW extra_columns AS
SELECT s.column_name
FROM source_schema s
LEFT JOIN target_schema t
  ON s.column_name = t.column_name
WHERE t.column_name IS NULL;


Result

In [0]:
%sql
select * from extra_columns;

Extract Extra Column Data

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW products_extra_data AS
SELECT
  discount,
  supplier_code
FROM databricks_catalog.practice.products_src;


In [0]:
%sql
select * from products_extra_data;

Store Extra Columns in Separate Container

In [0]:
%sql
    
CREATE OR REPLACE TABLE databricks_catalog.practice.products_extra_columns
USING DELTA
AS
SELECT * FROM products_extra_data;

In [0]:
%sql
    
select * from databricks_catalog.practice.products_extra_columns;

Load CLEAN Data into TARGET Table

In [0]:
%sql
INSERT INTO databricks_catalog.spractice.products_tgt
SELECT
  product_id,
  product_name,
  price,
  category,
  load_date
FROM databricks_catalog.practice.products_src;


In [0]:
%sql
select * from databricks_catalog.spractice.products_tgt;


Final Validation

In [0]:
%sql
SELECT * FROM databricks_catalog.spractice.products_tgt;


Extra columns stored safely


In [0]:
%sql
SELECT * FROM databricks_catalog.practice.products_extra_columns;
